In [82]:
import numpy as np
import cv2
from skimage import io
from skimage.util import img_as_ubyte
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

import matplotlib.pyplot as plt

from pathlib import Path
import json
import matplotlib.patches as patches
import time




In [83]:
#load video % config
img_list = []
stitch_calibration = np.load("./config/stitch_calibration.npy")
print(stitch_calibration)
with open('./config/rect_vertices_2_calib_samesize.json', 'r') as rect_file:
    rect = json.load(rect_file)

for i in range(0, 75):
    image = io.imread("./data/pics/img{}.jpg".format(i))
    # print(image.shape)
    image = cv2.resize(image, (800, 450))
    img_list.append(image)
    cv2.imshow("imread",image)
    cv2.waitKey()
img_list = np.asarray(img_list)
print(img_list.shape)
cv2.destroyAllWindows()

[['hor 0' '50']
 ['hor 1' '64']
 ['hor 2' '49']
 ['ver 0' '0']
 ['hor 0' '49']
 ['hor 1' '63']
 ['hor 2' '48']
 ['ver 1' '58']
 ['hor 0' '48']
 ['hor 1' '62']
 ['hor 2' '47']
 ['ver 2' '73']]
(75, 450, 800, 3)


In [84]:
# get image tiles
ncol = 4
nrow = 3
tiles_list = []

for s in range(0, img_list.shape[0]):
    frame = img_list[s]
#     print(frame.shape)
    video_tile = []
    for i in range(nrow):
        hstack = []
        for j in range(ncol):
            key = 'vertice{}'.format(i*ncol + j+1)
            left_top = rect['vertices'][key]['left_top']
            right_bottom = rect['vertices'][key]['right_bottom']

            tile = frame[int(left_top[1]):int(right_bottom[1]),
                        int(left_top[0]):int(right_bottom[0]), :]
            tile_flipped = cv2.flip(tile, -1)
            
            video_tile.append(tile_flipped)
#             if hstack == []:
#             hstack = tile_flipped
#             else:
#                 hstack = np.hstack((hstack, tile_flipped))
#         if vstack == []:
#             vstack = hstack
#         else:
#             vstack = np.vstack((vstack, hstack))
            
    tiles_list.append(np.asarray(video_tile, dtype = np.uint8))

tiles_list = np.asarray(tiles_list, dtype = np.uint8)
print(type(tiles_list), tiles_list.shape)



<class 'numpy.ndarray'> (75, 12, 80, 75, 3)


In [85]:
# def stitch

def find_ov(img1, img2, ov_pos, dim=1):
    if dim == 1:
        ov_1 = img1[:, -ov_pos:, :]
        ov_2 = img2[:,  :ov_pos, :]
    elif dim == 0:
        ov_1 = img1[-ov_pos:, :, :]
        ov_2 = img2[:ov_pos,  :, :]
    
    return ov_pos, ov_1, ov_2


def stitch(img1, img2, ov_pos, ov1, ov2, blend=True, dim=1): 
    
#     t1 = time.time()
    if blend:
        # linear blending
        ov1_blended = ov1.copy()
        ov2_blended = ov2.copy()
        for i in range(ov_pos):
            blend_coef = -(1/(ov_pos-1))*i + 1
            if dim==1:
                ov1_blended[:,i,:] = ov1[:,i,:]*blend_coef
                ov2_blended[:,i,:] = ov2[:,i,:]*(1.-blend_coef)
            elif dim==0:
                ov1_blended[i, :,:] = ov1[i, :,:]*blend_coef
                ov2_blended[i, :,:] = ov2[i, :,:]*(1.-blend_coef)
        ov_blended = ov1_blended + ov2_blended
        ov = ov_blended
    else:
        ov = ov1
#     print('time: linear blending >>>', time.time() - t1)
    
    if dim == 1:
        stitched_img = np.concatenate((img1[:, :-ov_pos, :], ov, img2[:, ov_pos: :]), axis = 1)
    elif dim == 0:
        stitched_img = np.concatenate((img1[:-ov_pos, :, :], ov, img2[ov_pos:, :, :]), axis = 0)
    
    return stitched_img



In [86]:
#stitch
%matplotlib inline

def save_image(image,addr,num):
    address = addr + str(num)+ '.jpg'
    cv2.imwrite(address,image)
    

stitched_list = []
img_stitched_height = None
change_sv = 1
s_mean = 28.0
v_mean = 150.0

for s in range(0, tiles_list.shape[0]):
    tile_list = tiles_list[s]
    print(tile_list.shape)
    for i in range(nrow):
        img_stitched_width = None
        for j in range(ncol):
            if j == 0:
                if change_sv == False: 
                    img_j0 = tile_list[i*4]
                else:
                    img_j0 = cv2.cvtColor(tile_list[i*4], cv2.COLOR_RGB2HSV)
                    img_j0 = img_j0.astype(np.float64)
                    img_j0[:,:,1] = np.clip(img_j0[:,:,1] * s_mean / np.mean(img_j0[:,:,1]),0.0,255.0)
                    img_j0[:,:,2] = np.clip(img_j0[:,:,2] * v_mean / np.mean(img_j0[:,:,2]),0.0,255.0)
                    img_j0 = cv2.cvtColor(img_j0.astype(np.uint8), cv2.COLOR_HSV2RGB)

                img_stitched_width = img_j0

            else:
                if change_sv == False: 
                    img_1 =  tile_list[i*4 + j-1]#tile_list_histMatched[i*4 + j-1]
                    img_2 =  tile_list[i*4 + j] #tile_list_histMatched[i*4 + j]
                else:
                    img_1 = cv2.cvtColor(tile_list[i*4 + j-1], cv2.COLOR_RGB2HSV)
                    img_1 = img_1.astype(np.float64)
                    img_1[:,:,1] = np.clip(img_1[:,:,1] * s_mean / np.mean(img_1[:,:,1]),0,255)
                    img_1[:,:,2] = np.clip(img_1[:,:,2] * v_mean / np.mean(img_1[:,:,2]),0,255)
                    img_1 = cv2.cvtColor(img_1.astype(np.uint8), cv2.COLOR_HSV2RGB)

                    img_2 = cv2.cvtColor(tile_list[i*4 + j], cv2.COLOR_RGB2HSV)
                    img_2 = img_2.astype(np.float64)
                    img_2[:,:,1] = np.clip(img_2[:,:,1] * s_mean / np.mean(img_2[:,:,1]),0,255)
                    img_2[:,:,2] = np.clip(img_2[:,:,2] * v_mean / np.mean(img_2[:,:,2]),0,255)
                    img_2 = cv2.cvtColor(img_2.astype(np.uint8), cv2.COLOR_HSV2RGB)
                ov_pos = int(stitch_calibration[i*4 + j-1][1])
#                 print(ov_pos)
                ov_pos, ov1, ov2 = find_ov(img_1, img_2, ov_pos, dim=1)
#                 print('{}th overlap size in row {}: '.format(j-1, i), ov_pos)

                # stitch images with stacking
                img_stitched_width = stitch(img_stitched_width, img_2, ov_pos, ov1, ov2, blend=True, dim=1)

#         print('Stitched row  width >>>  ', img_stitched_width.shape[1])
        if i == 0:
            img_stitched_height = img_stitched_width
            pass
        else:
            width = img_stitched_height.shape[1]
            width_new = img_stitched_width.shape[1]
            wdiff = width - width_new
#             print(i, wdiff)
            if wdiff > 0:
                img_stitched_width = np.hstack((img_stitched_width, img_stitched_width[:, -wdiff:]))
            elif wdiff < 0:
                img_stitched_width = img_stitched_width[:, :wdiff]
            ov_pos_h = int(stitch_calibration[(i+1)*4 - 1][1])
#             print(ov_pos_h)
            ov_pos_h, ov_h1, ov_h2 = find_ov(img_stitched_height, img_stitched_width, ov_pos_h, dim=0)

#             print('Row overlapping:', ov_pos_h)
            img_stitched_height = stitch(img_stitched_height, img_stitched_width, 
                                         ov_pos_h, ov_h1, ov_h2, blend=True, dim=0)

#     print(img_stitched_height.shape)
    save_image(img_stitched_height[:,:,::-1],'./output/p4/img',int(s))
    cv2.imshow("image", img_stitched_height[:,:,::-1])
    cv2.waitKey()
    print ("\r saving image {} ".format(int(s)), end="")
#     plt.figure(figsize=(6, 10))
#     plt.subplot(2,1,1)
#     plt.imshow(cv2.cvtColor(img_stitched_height,cv2.COLOR_RGB2GRAY), cmap='gray')
#     plt.subplot(2,1,2)
#     plt.imshow(img_stitched_height, cmap='gray')
cv2.destroyAllWindows()
print("\n done")

(12, 80, 75, 3)
 saving image 0 (12, 80, 75, 3)
 saving image 1 (12, 80, 75, 3)
 saving image 2 (12, 80, 75, 3)
 saving image 3 (12, 80, 75, 3)
 saving image 4 (12, 80, 75, 3)
 saving image 5 (12, 80, 75, 3)
 saving image 6 (12, 80, 75, 3)
 saving image 7 (12, 80, 75, 3)
 saving image 8 (12, 80, 75, 3)
 saving image 9 (12, 80, 75, 3)
 saving image 10 (12, 80, 75, 3)
 saving image 11 (12, 80, 75, 3)
 saving image 12 (12, 80, 75, 3)
 saving image 13 (12, 80, 75, 3)
 saving image 14 (12, 80, 75, 3)
 saving image 15 (12, 80, 75, 3)
 saving image 16 (12, 80, 75, 3)
 saving image 17 (12, 80, 75, 3)
 saving image 18 (12, 80, 75, 3)
 saving image 19 (12, 80, 75, 3)
 saving image 20 (12, 80, 75, 3)
 saving image 21 (12, 80, 75, 3)
 saving image 22 (12, 80, 75, 3)
 saving image 23 (12, 80, 75, 3)
 saving image 24 (12, 80, 75, 3)
 saving image 25 (12, 80, 75, 3)
 saving image 26 (12, 80, 75, 3)
 saving image 27 (12, 80, 75, 3)
 saving image 28 (12, 80, 75, 3)
 saving image 29 (12, 80, 75, 3)
 sav

In [40]:
print("\n done")


 done
